In [8]:
from task import Task
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
typename = 'ABCDEFGHIJKLMN'

In [9]:
def evaluate(method, df, dataset, sp, loop, nodeType = None, labelfile = None):    
    embeddings = df.loc[df[0].str.contains(nodeType)]
    embeddings.reset_index(drop=True, inplace=True)
    #labelfile = 'answersLabelAcception'
    #labelfile = 'answersLabelScore'
    label = np.loadtxt("dataset/{}/{}.txt".format(dataset, labelfile), dtype=np.int)
    print(label[:,1].max()+1)
    node2index = {}
    for i in range(len(embeddings)):
        node2index[embeddings[0][i]] = i
    # delete label -1 which means the truth is unknown
    delindices = np.where(label[:,1]==-1)[0]
    label = np.delete(label, delindices, axis=0)

    indicesList = []
    for i in range(len(label)):
        node = nodeType+str(label[i][0])
        if node not in node2index:
            print(node)
        assert node in node2index
        indicesList.append(node2index[node])
        label[i][0] = i
    embeddings.drop([df.columns[0]], axis=1,inplace=True) 
    embed = embeddings.values[indicesList]

    task = Task('CLF')
    task.classfication(embed, label, split_ratio=sp, loop=loop)

In [11]:
dataset = 'stackexchange-chemistry'
#dataset = 'airports'
graphid = 1
#methods = [ 'HAWE','hin2vec','node2bits','deepWalk','struc2vec','GraphSTONE']
#methods = ['CHAWE']
methods = ['HAWE']
#methods = ['RGCN', 'HGT']
for method in methods:
    df = pd.read_csv("embedding/{}-{}.embeddings".format(method, dataset), skiprows=1, header=None, sep=' ')  
    print(method)
    evaluate(method, df, dataset, 0.7, 50, nodeType = 'A', labelfile = 'answersLabelScore') #answersLabelScore #userReputation #airportsLabel

  0%|          | 0/50 [00:00<?, ?it/s]

HAWE
3


100%|██████████| 50/50 [00:06<00:00,  7.85it/s]

average performance. f1-micro|f1-macro: 0.4137 +/- 0.0177|0.4069 +/- 0.0172
